# 状態空間モデルとTransformer

## 最初の見取り図

状態空間モデルとTransformerでは、未来予測を使った意思決定を、実装可能な形に分解して学びます。

前提: 時系列モデリングと強化学習の基礎があると効果的です。

到達目標: 世界モデルを、状態表現・遷移・観測予測・計画の連鎖として理解し、実験設計へ落とし込める状態にします。

ここで扱う中心語は 「状態表現」、「遷移モデル」、「観測予測」、「計画」、「ロールアウト」、「状態空間モデルとTransformer」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは 世界モデル 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 手を動かす 1: 状態遷移の最小モデル

世界モデルの最初の核は遷移式です。状態と行動から次状態を作る操作を最小形で確認します。

In [ ]:
z_t = 0.25
a_t = 1.0
A, B = 0.92, 0.18
z_next = A * z_t + B * a_t
print('z_next =', round(z_next, 6))

遷移式の係数は環境のダイナミクス仮定を表します。係数の選び方で予測性質が大きく変わります。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 手を動かす 2: 観測予測を作る

次に、潜在状態から観測を復元する写像を作ります。状態推定と観測再現の役割分担をコードで掴みます。

In [ ]:
def decode(z):
    return {'position': 2.5 * z + 0.1, 'velocity': 0.8 * z - 0.05}
obs_next = decode(z_next)
print('obs_next =', {k: round(v, 4) for k, v in obs_next.items()})
print('keys =', list(obs_next.keys()))

観測予測を別関数に切ると、遷移誤差と観測誤差を分離して調整できます。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 数式メモ

1. z_{t+1} = f_theta(z_t, a_t)
2. o_hat_{t+1} = g_theta(z_{t+1})

## 手を動かす 3: ロールアウトを試す

ここで複数ステップ予測を実行します。1ステップでは見えない誤差累積を把握するためです。

In [ ]:
actions = [0.0, 1.0, 1.0, 0.0, -0.5]
z = 0.1
traj = []
for a in actions:
    z = 0.92 * z + 0.18 * a
    traj.append(round(z, 5))
print('rollout =', traj)

長期予測で崩れるなら、遷移モデルの安定性や状態表現の情報量不足を疑います。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 手を動かす 4: 計画候補を比較する

次に、複数の行動列を比較して、どの計画が望ましいかを評価します。モデルベース強化学習の中心操作です。

In [ ]:
plans = [[0, 1, 1], [1, 1, 1], [0, 0, 1]]
def score_plan(plan):
    z = 0.1
    for a in plan:
        z = 0.92 * z + 0.18 * a
    return z
scores = [round(score_plan(p), 5) for p in plans]
print('scores =', scores)

計画評価が可能になると、実環境での試行回数を抑えた探索がしやすくなります。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 手を動かす 5: モデル誤差を監視する

最後に、予測と実測の差を定量化します。世界モデルは『予測できる範囲』を常に点検する運用が重要です。

In [ ]:
pred = [0.10, 0.22, 0.31, 0.29]
real = [0.11, 0.25, 0.28, 0.35]
errors = [abs(p - r) for p, r in zip(pred, real)]
print('errors =', [round(e, 4) for e in errors])
print('mean_error =', round(sum(errors) / len(errors), 5))

平均誤差だけでなく時点別誤差を追うと、どの遷移条件でモデルが弱いかを特定しやすくなります。

この節では、状態表現 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## この節の要点

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 1ステップ誤差だけで安心してしまう
2. 長期予測の誤差爆発を監視しない
3. 状態表現が不足しているのにモデルだけ調整する

次は「観測予測モデル（自己回帰・マスク予測・拡散）」へ進み、今回のコードと何が変わるかを比較してください。